# Sequencer instructions

## Single channel

The module `core.device.sequencer.instructions` provides instructions that can be used to generate complex time sequences. 

The compilation algorithm generates such instructions based on the user input. 
The instructions are then sent to the sequencer devices whose only role is to program these instructions in the hardware.

The basic building block is a `Pattern` that can be used to explicitly represent an arbitrary sequence of values to be output on a sequencer for each timestep.

In [1]:
import matplotlib.pyplot as plt

from core.device.sequencer.instructions import Pattern, convert_to_change_arrays

pattern = Pattern([True, False, True, True, False])


def plot_instruction(instruction):
    times, values = convert_to_change_arrays(instruction)
    plt.plot(times, values, drawstyle="steps-post")
    plt.xlabel("Time [ticks]")
    plt.ylabel("Value")


plot_instruction(pattern)

Patterns can be combined by concatenating or repeating them, which allows to build complex instructions easily.

In [2]:
instr_1 = (
    20 * Pattern([True])
    + 30 * Pattern([False])
    + 4 * (5 * Pattern([True]) + 5 * Pattern([False]))
    + 10 * Pattern([False])
    + 15 * Pattern([True])
    + 15 * Pattern([False])
    + 6 * Pattern([True, True, False])
    + 30 * Pattern([False])
)

plot_instruction(instr_1)

All subclasses of `SequencerInstruction` (`Pattern`, `Concatenate` and `Repeat`) overloads `+` and `*` operators to easily combine instructions. 

Combining instructions with these operators will build a tree of operations but will never evaluate it unless explicitly asked.

In [3]:
instr_2 = 7 * Pattern([True]) + 3 * Pattern([False])
instr_2

If one wants to convert the tree instruction to a flat representation, they can use the method `.to_pattern()`. 

In [4]:
instr_2.to_pattern()

It is however often much more efficient to work with the tree representation because it allows to represent instructions with many time steps very compactly.

## Combining channels

Using the above techniques, it is possible to construct complex time sequence for a single output channel of a sequencer.

The compilation algorithm will compute an instruction for each channel of a given sequencer device.
However, it is difficult to program them directly to the device, because the channels can have different operation trees.

For example, one might want to have a clock with a periodicity of 2 ticks on the first channel, a clock with a periodicity of 3 ticks on the second one and just a single on/off toggle on the third one:

In [5]:
clock_0 = Pattern([True, False]) * 30
clock_1 = Pattern([True, True, False]) * 20

toggle = Pattern([True]) * 35 + Pattern([False]) * 25

print(clock_0.as_type(int))
print(clock_1.as_type(int))
print(toggle.as_type(int))

The function `stack_instructions` can be used to find a common tree representation of multiple instructions having all the same length:

In [6]:
from core.device.sequencer.instructions import stack_instructions, with_name

stacked = stack_instructions(
    [
        with_name(clock_0, "clock 0"),
        with_name(clock_1, "clock 1"),
        with_name(toggle, "toggle"),
    ]
)

print(stacked["clock 0"].as_type(int))
print(stacked["clock 1"].as_type(int))
print(stacked["toggle"].as_type(int))

Note: `stack_instructions` will always return a result where each field is equivalent to the one passed in argument, but it might not compute the most compact representation. 
In the worst cases, this function may flatten completely the instruction trees passed.

## Walking the tree

A common operation is to execute a function for each node of the tree recursively.
An approach that works in most cases it to use pattern matching with `match`:

In [12]:
from core.device.sequencer.instructions import (
    Concatenate,
    Pattern,
    Repeat,
    SequencerInstruction,
)


def number_operations(instruction: SequencerInstruction) -> int:
    """Counts the number of + or * in an instruction"""

    match instruction:
        case Pattern():
            return 0
        case Concatenate(instructions):
            return (
                len(instructions)
                - 1
                + sum(number_operations(instruction) for instruction in instructions)
            )
        case Repeat(repetitions=repetitions, instruction=repeated):
            return 1 + number_operations(repeated)


number_operations(stacked)